In [ ]:
import itertools

import matplotlib.pyplot as plt
import numba
import numpy as np
from scipy import stats 

# Análisis estadístico

## Introducción a las pruebas de hipótesis
Supongamos que se nos da una moneda y al tirarla 30 veces obtenemos 22 soles y 8 águilas. ¿La moneda es justa?
- *Hipótesis nula*: El resultado observado se debe al azar.
- *Hipótesis alternativa*: La negación de la hipótesis nula.

La probabilidad de obtener $k$ soles en $n$ volados donde la probabilidad de obtener sol en un volado es $p$, está dada por:
$$\mathrm{P}(X=k)=\binom{n}{k} \, p^k \, (1 - p)^{n - k}$$

*Demostración*. Una sucesión de $n$ volades se puede representar como una sucesión de letras $A$/$S$ (águila o sol). Por ejemplo,
$$ASAASSASAAA$$
- Si en $n$ volados ocurre $k$ veces sol, es equivalente a seleccionar $k$ símbolos de la sucesión y rellenarlos con $S$ y el resto con $A$. Esto se puede hacer en $\binom{n}{k}$ formas distintas.
- La probabilidad de cada sucesión de $k$ soles en $n$ volados es la probabilidad de obtener $k$ soles y $n -k$ águilas; es decir $p^k\,(1- p)^{n - k}$.

Por lo tanto la probabilidad total de obtener $k$ soles (en cualquier orden) en $n$ volados es $\binom{n}{k} \, p^k \, (1 - p)^{n - k}$. $\blacksquare$

In [ ]:
n, r = (20, 7)
L = list(range(n, n - r, -1))
assert len(L) == r
assert L[0] == n
assert L[-1] == n - r + 1

In [ ]:
def permutaciones_sin_repeticion(n, r):
    accum = 1
    for x in range(n, n - r, -1):
        accum *= x
    return accum

def factorial(n):
    accum = 1
    for x in range(2, n + 1):
        accum *= x
    return accum

def combinaciones(n, k):
    return permutaciones_sin_repeticion(n, k)//factorial(k)

def probabilidad(n, k, p):
    return combinaciones(n, k) * p**k * (1 - p)**(n - k)

In [ ]:
import fractions
P = probabilidad(30, 22, fractions.Fraction(1/2))
print('La probabilidad de 22 soles en 30 volados es', P,', o aprox.', float(P))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

n, p = (30, 1/2)
df = pd.DataFrame({
    'Cantidad de soles': list(range(31)),
    'Probabilidad': [probabilidad(n, k, p) for k in range(31)]})
df.head()

In [ ]:
df.plot(x='Cantidad de soles', y='Probabilidad', kind='scatter')

Probabilidad de ver entre 9 y 21 soles en 30 volados:

In [ ]:
df[(df['Cantidad de soles'] >= 9) & (df['Cantidad de soles'] <= 21)]['Probabilidad'].sum()

Probabilidad de NO ver entre 9 y 21 volados:

In [ ]:
1 - _

La probabilidad de observar estos datos con la hipótesis nula $H_0$ se le llama el **p-valor**.
- Rechazamos la hipótesis nula cuando el $p$-valor es muy pequeño.
- No rechazamos la hipótesis nula cuando el $p$-valor está sobre cierto umbral.

Prueba de hipótesis| Aceptar $H_0$ | Rechazar $H_0$
-------------------|---------------|---------------
$H_0$ es verdadera |     ✓         | Error tipo I (falso positivo)
$H_0$ es falsa     | Error tipo II (falso negativo) | ✓

<img src="https://effectsizefaq.files.wordpress.com/2010/05/type-i-and-type-ii-errors.jpg" alt="Errores de prueba de hipótesis" width="50%"/>

## Método alternativo: simulación

In [ ]:
@numba.jit
def calcular_p(n, p, repeticiones=1_000_000):
    M = 0
    for i in range(repeticiones):
        k = np.sum(np.random.random(size=n) < p)
        M += int((k > 21) or (k < 8))
    return M / repeticiones

In [ ]:
calcular_p(30, 0.5)

In [ ]:
calcular_p(30, 0.5)

## Prueba de permutaciones aleatorias para la diferencia de medias

In [ ]:
X = [1.1, 3.0, 2.8, 1.9, 2.5, 2.6]
Y = [8.1, 3.5, 5.4, 2.4, 3.0, 4.1]

In [ ]:
def comparar_normales(X, Y):
    N_1 = stats.norm(np.mean(X), np.std(X))
    N_2 = stats.norm(np.mean(Y), np.std(Y))

    a, b = min(itertools.chain(X, Y)), max(itertools.chain(X, Y))
    dominio = np.linspace(a, b)
    plt.plot(dominio, N_1.pdf(dominio), color='blue')
    plt.plot(dominio, N_2.pdf(dominio), color='red')
comparar_normales(X, Y)

In [ ]:
X = np.random.normal(loc=8, scale=4, size=30)
Y = np.random.normal(loc=8, scale=4, size=40)
comparar_normales(X, Y)

In [ ]:
X

In [ ]:
Y

In [ ]:
np.mean(X) - np.mean(Y)

In [ ]:
@numba.jit
def prueba_diferencia_medias(X, Y, repeticiones=1_000_000):
    Z = np.concatenate([X, Y])
    diferencia_obs = np.mean(X) - np.mean(Y)
    diferencias = []
    for i in range(repeticiones):
        np.random.shuffle(Z)
        X_simulado = Z[:len(X)]
        Y_simulado = Z[len(X):]
        diferencias.append(np.mean(X_simulado) - np.mean(Y_simulado))
    plt.hist(diferencias, bins=30)
    plt.axvline(diferencia_obs, color='red')
    return np.sum(np.abs(np.array(diferencias)) >= abs(diferencia_obs))/repeticiones

In [ ]:
prueba_diferencia_medias(X, Y)

## Intervalos de confianza para la media

In [ ]:
N = np.random.randint(1, 50, size=20)
xbar = [None]*10000
for i in range(10000):
    sample = N[np.random.randint(len(N), size=len(N))]
    xbar[i] = np.mean(sample)

plt.hist(xbar, bins=30)

In [ ]:
conf = 95
a = (100 - conf)/2
b = 100 - a
np.percentile(xbar, [a, b])

In [ ]:
np.mean(N)